In [4]:

import re
import time
import random
import requests
import numpy as np
import pandas as pd
import _pickle as pickle
from selenium import webdriver
from bs4 import BeautifulSoup as bs
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys

In [5]:
biolist = []
name_list = []
age_list = []
# occupation_list = []


A lot of website are providing fake data geneartion tools, here are the list of a few website.
1. https://www.fakepersongenerator.com/Index/generate 
2. https://randomuser.me/documentation#format 
3. https://www.character-generator.org.uk/bio/ 
4. https://www.dating-profile-generator.org.uk/create.php?type=1 
Webscraping code depends upon website. I will be using https://www.character-generator.org.uk/bio/ in my code


In [6]:
URL = "https://www.character-generator.org.uk/bio/"

In [9]:
# launch WebDriver instance
driver = webdriver.Chrome()

# launch website
driver.get(URL)

# find required element and perform action 
driver.find_element_by_id('fill_all').click()
driver.find_element_by_id('quick_submit').click()

#wait for reponse
time.sleep(2)

# get new url
url = driver.current_url

#get conents from url
page = requests.get(url)
soup = bs(page.content)
biostr=''



try:
    # Getting the bios
    bios = soup.find('div', class_='bio').find_all('p')

    # Adding to a list of the bios
    for i in bios:
        biostr += re.sub('<p>|</p>','',str(i))
        
        # To collect name, age from first string
        name, age = ' '.join(biostr.split('.')[0].split(' ')[0:3]),int(re.findall("\d+", biostr.split('.')[0])[0])
        
        # s = biostr.split('.')[0]
        # occupation = s[s.find("-old ")+ len("-old "):s.find(" who")]

    biolist.append(biostr)
    name_list.append(name)
    age_list.append(age)
    # occupation_list.append(occupation)
    
    # close the instance
    driver.close()
except:
    pass

len(biolist)

2

In [10]:
# Zip all three list and create a dataframe
profile = pd.DataFrame(zip(name_list, age_list,biolist), columns=['name','age','bios'])

In [11]:
profile

,name,age,bios
0,Richard Luke Trescothik,21,Richard Luke Trescothik is a 21-year-old trade...
1,Mark Rick Sweet,29,Mark Rick Sweet is a 29-year-old theatre actor...


We can automate this process for gathering large number of fake profiles data

In [ ]:
# Randomizing the refresh rate
seq = [i/10 for i in range(8,18)]


# Gathering bios by looping and refreshing the web page
for _ in tqdm(range(1000)):
    
    # launch WebDriver instance
    driver = webdriver.Chrome()

    # launch website
    driver.get(URL)
    # find required element and perform action 
    driver.find_element_by_id('fill_all').click()
    driver.find_element_by_id('quick_submit').click()
    #wait for reponse
    time.sleep(2)
    # get new url
    url = driver.current_url
    #get conents from url
    page = requests.get(url)
    soup = bs(page.content)
    biostr=''

    try:
        # Getting the bios
        bios = soup.find('div', class_='bio').find_all('p')

        # Adding to a list of the bios
        for i in bios:
            biostr += re.sub('<p>|</p>','',str(i)) 
            
            name, age = ' '.join(biostr.split('.')[0].split(' ')[0:3]),int(re.findall("\d+", biostr.split('.')[0])[0])
        
            # s = biostr.split('.')[0]
            # occupation = s[s.find("-old ")+ len("-old "):s.find(" who")]

        biolist.append(biostr)
        name_list.append(name)
        age_list.append(age)

        # close the instance
        driver.close()
        print(len(biolist))
    except:
        pass

    
    # Sleeping 
    time.sleep(random.choice(seq))

In [ ]:
len(biolist)

In [ ]:
profile = pd.DataFrame(zip(name_list, age_list,biolist), columns=['name','age','bios'])

In [ ]:
profile

Finally we can serialise created datframe object for future use

In [12]:

with open("bios_data0.pkl", "wb") as fp:
    pickle.dump(profile, fp)